In [1]:
#  Imports

from src.parsers import HMParser
from src.repairs import PymooHMRepair
from src.scenes import PymooHMProblem

from src.metrics import CommunityMetrics

import numpy as np

# ignore the warnings (only demonstration purposes!)
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Data parsing

data = HMParser(file_path='data/EC_V4.xlsx', ec_id=1)
data.parse()

In [3]:
# Pymoo imports

from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize
from pymoo.termination.default import DefaultSingleObjectiveTermination

algorithm = DE(pop_size=30,
               sampling=LHS(),
               variant="DE/rand/1/bin",
               CR=0.3,
               dither="vector",
               jitter=False,
               repair=PymooHMRepair(data=data))


# Default termination:
termination = DefaultSingleObjectiveTermination(xtol=1e-8,
                                                cvtol=1e-6,
                                                ftol=1e-6,
                                                period=1000,
                                                n_max_gen=2e1,
                                                n_max_evals=5e9)

customProblem = PymooHMProblem(data=data)

temp_res = minimize(customProblem,
                    algorithm,
                    termination=termination,
                    save_history=False,
                    verbose=True)

30
n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       30 |  3.238264E+05 |  3.291137E+03
30
     2 |       60 |  2.837603E+05 |  3.291137E+03
30
     3 |       90 |  2.370138E+05 |  3.291137E+03
30
     4 |      120 |  1.969193E+05 |  3.021715E+03
30
     5 |      150 |  1.701567E+05 |  3.021715E+03
30
     6 |      180 |  1.434055E+05 |  3.021715E+03
30
     7 |      210 |  1.266946E+05 |  3.021715E+03
30
     8 |      240 |  1.099102E+05 |  2.921541E+03
30
     9 |      270 |  9.655241E+04 |  2.687797E+03
30
    10 |      300 |  7.652225E+04 |  2.687797E+03
30
    11 |      330 |  6.646244E+04 |  2.594961E+03
30
    12 |      360 |  4.971762E+04 |  2.594961E+03
30
    13 |      390 |  3.296820E+04 |  2.594961E+03
30
    14 |      420 |  2.622627E+04 |  2.553485E+03
30
    15 |      450 |  2.281901E+04 |  2.477938E+03
30
    16 |      480 |  1.610890E+04 |  2.427720E+03
30
    17 |      510 |  9.379268E+03 |  2.420087E+03
30
    18 |      540 |  2.653790E+03 |  2.282128E

In [4]:
# Obj function value

temp_res.F

array([2282.12834025])

In [5]:
# Solution

decoded_resources = customProblem.decode(temp_res.X)

In [6]:
# Show the decoded resources

decoded_resources

{'genActPower': [genActPower_00,
  genActPower_01,
  genActPower_02,
  genActPower_03,
  genActPower_04,
  genActPower_05,
  genActPower_06],
 'genExcActPower': [genExcActPower_00,
  genExcActPower_01,
  genExcActPower_02,
  genExcActPower_03,
  genExcActPower_04,
  genExcActPower_05,
  genExcActPower_06],
 'pImp': pImp,
 'pExp': pExp,
 'loadRedActPower': [loadRedActPower_00,
  loadRedActPower_01,
  loadRedActPower_02,
  loadRedActPower_03,
  loadRedActPower_04,
  loadRedActPower_05],
 'loadCutActPower': [loadCutActPower_00,
  loadCutActPower_01,
  loadCutActPower_02,
  loadCutActPower_03,
  loadCutActPower_04,
  loadCutActPower_05],
 'loadENS': [loadENS_00,
  loadENS_01,
  loadENS_02,
  loadENS_03,
  loadENS_04,
  loadENS_05],
 'loadActPower': [loadActPower_00,
  loadActPower_01,
  loadActPower_02,
  loadActPower_03,
  loadActPower_04,
  loadActPower_05],
 'storDchActPower': [storDchActPower_00,
  storDchActPower_01,
  storDchActPower_02],
 'storChActPower': [storChActPower_00, storCh

In [7]:
# Metrics

metric = CommunityMetrics()
print(metric)

Community Metrics available: 
- self_consumption
- total_produced
- total_consumed
- import_export_balance



In [8]:
# Check the metrics
metric.cmd_self_consumption(production=decoded_resources['genActPower'],
                            consumption=decoded_resources['loadActPower'])

1.6052307283912177

In [9]:
# Do a list of metrics at once

metric.check_metrics_2inputs(components_a=decoded_resources['genActPower'],
                             components_b=decoded_resources['loadActPower'],
                             metric_list=['self_consumption'])

{'self_consumption': 1.6052307283912177}

In [10]:
# Try to get an error
# Total consumption is not a valid metric with two components

metric.check_metrics_2inputs(components_a=decoded_resources['genActPower'],
                             components_b=decoded_resources['loadActPower'],
                             metric_list=['self_consumption',
                                          'total_consumed'])

The requested metric is not available with provided inputs:  total_consumed


{'self_consumption': 1.6052307283912177, 'total_consumed': None}

In [11]:
# Do note that the following metrics do not make sense to be called like this.
# Use at own discretion

metric.check_metrics_1input(decoded_resources['loadActPower'],
                            metric_list=['total_consumed'])

{'total_consumed': 342.4098855999999}

In [14]:
# Calculate the self sufficiency of the community

production_list = ['genActPower', 'loadCutActPower', 'loadRedActPower', 'loadENS',
                   'storDchActPower', 'v2gDchActPower']

consumption_list = ['genExcActPower', 'loadActPower', 'storChActPower',
                    'v2gChActPower']

# Append all the production values to a single array. Need to go a level deeper
community_productions = np.array([decoded_resources[prod][prod_elem]
                                  for prod in production_list
                                  for prod_elem in range(len(decoded_resources[prod]))])
community_productions = np.append(community_productions, decoded_resources['pImp'])

# Same for the consumption values
community_consumptions = np.array([decoded_resources[cons][cons_elem]
                                   for cons in consumption_list
                                   for cons_elem in range(len(decoded_resources[cons]))])
community_consumptions = np.append(community_consumptions, decoded_resources['pExp'])

print('Self consumption: ', metric.cmd_self_consumption(community_productions, community_consumptions))
print('Cost: ', metric.cmd_energy_costs(community_productions[:-1], community_consumptions[:-1]))

Self consumption:  0.9999999999999998
Cost:  -366.6457024117808
